In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
import warnings
warnings.filterwarnings('ignore')
   

### 1. Loading the datasets and showing summary 

In [3]:
df_patient = pd.read_csv('Patient_Data.csv')
df_patient.head()

,PatientID,Name,Department,Doctor,BillAmount,ReceptionistID,CheckInTime
0,101,Alice,Cardiology,Dr. Smith,5000.0,1,2023-01-10 09:00
1,102,Bob,Neurology,Dr. John,NaN,2,2023-01-11 10:30
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,1,2023-01-12 11:00
3,104,David,Cardiology,Dr. Smith,6200.0,3,2023-01-13 12:00
4,105,Eva,Dermatology,Dr. Rose,NaN,2,2023-01-14 08:45


In [4]:
df_patient.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   PatientID       6 non-null      int64  
 1   Name            6 non-null      object 
 2   Department      6 non-null      object 
 3   Doctor          6 non-null      object 
 4   BillAmount      4 non-null      float64
 5   ReceptionistID  6 non-null      int64  
 6   CheckInTime     6 non-null      object 
dtypes: float64(1), int64(2), object(4)
memory usage: 468.0+ bytes


In [5]:
df_billing = pd.read_csv('Billing_Data.csv')
df_billing.head()

,PatientID,InsuranceCovered,FinalAmount
0,101,2000,3000
1,102,1500,3500
2,103,2500,5000
3,104,3000,3200
4,105,1000,4000


In [6]:
df_billing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   PatientID         5 non-null      int64
 1   InsuranceCovered  5 non-null      int64
 2   FinalAmount       5 non-null      int64
dtypes: int64(3)
memory usage: 252.0 bytes


In [7]:
df_patient['CheckInTime'] = pd.to_datetime(df_patient['CheckInTime'], format='%Y-%m-%d %H:%M')   

### 2. Column Selection and Dropping

In [8]:
df_patient_new = df_patient.drop(columns=['ReceptionistID', 'CheckInTime'])

In [9]:
df_patient_new.head()

,PatientID,Name,Department,Doctor,BillAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0
1,102,Bob,Neurology,Dr. John,NaN
2,103,Charlie,Orthopedics,Dr. Lee,7500.0
3,104,David,Cardiology,Dr. Smith,6200.0
4,105,Eva,Dermatology,Dr. Rose,NaN


In [10]:
df_patient_new.isnull().sum()

PatientID     0
Name          0
Department    0
Doctor        0
BillAmount    2
dtype: int64

### 3. Handling Null Values effecently with MICE(Multiple Imputation by Chained Equations) method insted of replacing it with mean

```We can see we have null values in our data set , Since Bill Amount is an essential column we are applying **MICE** Technique handle accurately and avoide skewness of the data ```

In [11]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

In [12]:
imputer = IterativeImputer(random_state=0, max_iter=10)
imputed_array = imputer.fit_transform(df_patient_new[['BillAmount']])

df_patient_new['BillAmount'] = np.round(imputed_array[:, 0])

In [13]:
df_patient_new.isnull().sum()

PatientID     0
Name          0
Department    0
Doctor        0
BillAmount    0
dtype: int64

### 4. Merging Patients dataset and Billing Data Set

In [14]:
df_final = pd.merge(df_patient_new, df_billing, on='PatientID', how='inner')
df_final

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0,2000,3000
1,102,Bob,Neurology,Dr. John,5925.0,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.0,3000,3200
4,105,Eva,Dermatology,Dr. Rose,5925.0,1000,4000
5,101,Alice,Cardiology,Dr. Smith,5000.0,2000,3000


##### Checking for duplicate values

In [15]:
df_final.duplicated().sum()

np.int64(1)

### 5. droppinig dupicate rows

In [16]:
df_final = df_final.drop_duplicates()
df_final

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0,2000,3000
1,102,Bob,Neurology,Dr. John,5925.0,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.0,3000,3200
4,105,Eva,Dermatology,Dr. Rose,5925.0,1000,4000


### 6 .Finding Total bill per department using **GroupBy**

In [17]:
total_bill_per_dept = df_final.groupby('Department')['BillAmount'].sum().reset_index()
total_bill_per_dept

,Department,BillAmount
0,Cardiology,11200.0
1,Dermatology,5925.0
2,Neurology,5925.0
3,Orthopedics,7500.0


### 7. Concatenation of New Data (column   wise)

##### Since we already had columns named "InsuranceCovered " and  "FinalAmount " , we are concatenatind with new meaningful columns using feature engineering

In [18]:
num_rows = len(df_final)
print("--- A. Creating New Meaningful Columns (Feature Engineering) ---")
los_data = np.random.randint(1, 15, size=num_rows)

--- A. Creating New Meaningful Columns (Feature Engineering) ---


In [19]:
status_data = np.random.choice(['Paid', 'Pending', 'Partial'], size=num_rows, p=[0.6, 0.3, 0.1])

df_new_features = pd.DataFrame({
'LengthOfStay': los_data,
'PaymentStatus': status_data
})

In [20]:
df_final_full = pd.concat([df_final.reset_index(drop=True), df_new_features.reset_index(drop=True)], axis=1)


In [21]:
df_final_full['RevenuePerDay'] = df_final_full['BillAmount'] / df_final_full['LengthOfStay']
print("\nNew Features Added Successfully:")
df_final_full.head()
print("-" * 50)
print("Final Cleaned and Integrated Dataset (Head of all new/key columns):")



New Features Added Successfully:
--------------------------------------------------
Final Cleaned and Integrated Dataset (Head of all new/key columns):


In [22]:
df_final

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount
0,101,Alice,Cardiology,Dr. Smith,5000.0,2000,3000
1,102,Bob,Neurology,Dr. John,5925.0,1500,3500
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,2500,5000
3,104,David,Cardiology,Dr. Smith,6200.0,3000,3200
4,105,Eva,Dermatology,Dr. Rose,5925.0,1000,4000


In [23]:
df_final_full.head()

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount,LengthOfStay,PaymentStatus,RevenuePerDay
0,101,Alice,Cardiology,Dr. Smith,5000.0,2000,3000,14,Paid,357.142857
1,102,Bob,Neurology,Dr. John,5925.0,1500,3500,8,Pending,740.625000
2,103,Charlie,Orthopedics,Dr. Lee,7500.0,2500,5000,5,Pending,1500.000000
3,104,David,Cardiology,Dr. Smith,6200.0,3000,3200,1,Paid,6200.000000
4,105,Eva,Dermatology,Dr. Rose,5925.0,1000,4000,6,Partial,987.500000


### 8. Concatenation of New Data (Row wise)

##### Creating new patients data 

In [24]:
df_new_patients = pd.DataFrame({
'PatientID': [106, 107, 108], 
'Name': ['Frank', 'Grace', 'Henry'],
'Department': ['Pediatrics', 'Cardiology', 'Neurology'],
'Doctor': ['Clark', 'Patel', 'Green'],
'BillAmount': [12000.0, 60000.0, 28000.0],
'InsuranceCovered': [8000.0, 45000.0, 18000.0],
'FinalAmount': [4000.0, 15000.0, 10000.0],
'LengthOfStay': [2, 10, 4],
'PaymentStatus': ['Paid', 'Pending', 'Paid']
})
df_new_patients

,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount,LengthOfStay,PaymentStatus
0,106,Frank,Pediatrics,Clark,12000.0,8000.0,4000.0,2,Paid
1,107,Grace,Cardiology,Patel,60000.0,45000.0,15000.0,10,Pending
2,108,Henry,Neurology,Green,28000.0,18000.0,10000.0,4,Paid


In [25]:
# Calculate RevenuePerDay for the new data
df_new_patients['RevenuePerDay'] = df_new_patients['BillAmount'] / df_new_patients['LengthOfStay']
print(f"Row Count Before Concatenation: {len(df_final_full)}")

Row Count Before Concatenation: 5


In [26]:
df_final_full = pd.concat([ df_final_full , df_new_patients] , ignore_index=True)


In [27]:
print(f"Row Count After Adding New Patients: {len(df_final_full)}")
df_final_full.tail()

Row Count After Adding New Patients: 8


,PatientID,Name,Department,Doctor,BillAmount,InsuranceCovered,FinalAmount,LengthOfStay,PaymentStatus,RevenuePerDay
3,104,David,Cardiology,Dr. Smith,6200.0,3000.0,3200.0,1,Paid,6200.0
4,105,Eva,Dermatology,Dr. Rose,5925.0,1000.0,4000.0,6,Partial,987.5
5,106,Frank,Pediatrics,Clark,12000.0,8000.0,4000.0,2,Paid,6000.0
6,107,Grace,Cardiology,Patel,60000.0,45000.0,15000.0,10,Pending,6000.0
7,108,Henry,Neurology,Green,28000.0,18000.0,10000.0,4,Paid,7000.0


In [28]:
df_final_full.describe()

,PatientID,BillAmount,InsuranceCovered,FinalAmount,LengthOfStay,RevenuePerDay
count,8.00000,8.000000,8.000000,8.000000,8.000000,8.000000
mean,104.50000,16318.750000,10125.000000,5962.500000,6.250000,3598.158482
std,2.44949,19226.055377,15191.045295,4294.161318,4.301163,2922.016842
min,101.00000,5000.000000,1000.000000,3000.000000,1.000000,357.142857
25%,102.75000,5925.000000,1875.000000,3425.000000,3.500000,925.781250
50%,104.50000,6850.000000,2750.000000,4000.000000,5.500000,3750.000000
75%,106.25000,16000.000000,10500.000000,6250.000000,8.500000,6050.000000
max,108.00000,60000.000000,45000.000000,15000.000000,14.000000,7000.000000


In [29]:
df_final_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8 entries, 0 to 7
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   PatientID         8 non-null      int64  
 1   Name              8 non-null      object 
 2   Department        8 non-null      object 
 3   Doctor            8 non-null      object 
 4   BillAmount        8 non-null      float64
 5   InsuranceCovered  8 non-null      float64
 6   FinalAmount       8 non-null      float64
 7   LengthOfStay      8 non-null      int64  
 8   PaymentStatus     8 non-null      object 
 9   RevenuePerDay     8 non-null      float64
dtypes: float64(4), int64(2), object(4)
memory usage: 772.0+ bytes


In [30]:
df_final_full.duplicated().sum()

np.int64(0)

In [32]:
df_final_full.isnull().sum()

PatientID           0
Name                0
Department          0
Doctor              0
BillAmount          0
InsuranceCovered    0
FinalAmount         0
LengthOfStay        0
PaymentStatus       0
RevenuePerDay       0
dtype: int64

##### Exporting Our final Dataset to perform further analytics on department-wise revenue or doctor performance

In [35]:
df_final_full.to_csv('patient_billing.csv', sep='\t', na_rep='MISSING', index=False)

### Summary / Conclution

This comprehensive data wrangling process successfully executed all required tasks efficiently, focusing on a smarter data integration approach.



* **Data Integration:** The final dataset was assembled via **row-wise concatenation** (adding new patients) and enhanced via **column-wise concatenation** of new features.


* **Feature Engineering (New Task):** We successfully added meaningful columns—**LengthOfStay** (key metric), **PaymentStatus** (categorical), and **RevenuePerDay** (derived metric)—to significantly enhance the dataset's analytical depth.


* **Analytical Readiness:** The final **df_final_full** DataFrame is structured to allow for complex analytics, such as assessing the correlation between Length of Stay and Revenue per Day, or analyzing revenue by Payment Status.